In [1]:
import gym
import numpy as np
from IPython.display import clear_output
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
env = gym.make('CartPole-v1')       # creates CartPole environment
new_state = env.reset()             # env.reset() resets the cartpole and returns the new state
done =   False                      # whether the simulation of one episode has completed on not
actions = [0, 1]                    # list of possible actions : 0 corresponds to "pushing the cart to the left" and 1 correspinds to "pushing the cart to the left"

discount = 0.97                     # discount rate
epsilon = 1                         # probability of exploring other possibilities instead of following the policy (we will be decreasing this value with every iteration)


In [3]:
# random normal initialization of 2*4 matrix W: 
W = tf.get_variable("W", [2,4], initializer = tf.random_normal_initializer(stddev = 0.1), dtype=tf.float64)   
# create a placeholder for new_state, which we will be feeding every iteration: 
New_state = tf.placeholder(tf.float64, shape = (4,1))   
# Multiply W and New_state to obtain Q vector   ( or Q(s) ) :
Q_s = tf.matmul(W, New_state)  
# We will be taking the action with maximum Q value, Q_sa is the Q value associated with the action
Action = tf.math.argmax(Q_s) 
Q_sa = tf.reduce_max(Q_s)
# We pass this action to the environment and it will return us reward and the next state
# So, create a placeholder for reward and next state, which we will be feeding every iteration: 
Reward = tf.placeholder(tf.float64, shape = (1,1))
Next_state = tf.placeholder(tf.float64, shape = (4,1))
#Bellman equation: Current_Q_value =  Reward + future Q value
# We expect our W to predict this
Target = Reward + discount * tf.reduce_max(tf.matmul(W, Next_state))
#squared error is our cost
Cost = tf.square(Target - Q_sa)
train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(Cost)

In [4]:
# start tensorlow session and initialize global variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# in case you want to save the best working W
best_reward = -np.Inf
best_W1 = sess.run(W)
# number of episodes of simulation
Episodes = 500

In [5]:
for episode in range(Episodes):
    total_reward = 0           # to keep track of total_reward in this episode
    done = False               # initialize every episode with done = False
    new_state = env.reset()    # reset the environment every episode
    if episode%49==0:          # show/render environment only after every 49 episodes, to speed up the training  (yeah i am odd)
        show = True
    else:
        show = False
    while not done:
        # feed the placeholder New_state with new_state to get Action
        action  = sess.run(Action, feed_dict = {New_state:new_state.reshape((4,1))})      
        state = new_state         
        # two possible actions : random_action with probability of epsilon, and q_action with probabilty of 1-epsilon
        random_action = np.random.choice(actions)
        q_action = actions[action[0]]
        action_that_gets_passed = np.array([random_action if np.random.rand() < epsilon else q_action])
        # pass action_that_gets_passed to update the state and get some reward!
        new_state, reward, done, _ = env.step(action_that_gets_passed[0])
        reward = reward if not done else -reward   # because this env gives positive reward even when simulation terminates
        total_reward += reward              # total _reward in this episode
        # feed values to respective placeholders and compute train
        feed = {Action:action_that_gets_passed, Reward: np.array(reward).reshape(1,1), New_state:state.reshape((4,1)), Next_state:new_state.reshape((4,1))}
        sess.run([train, Cost], feed_dict = feed) 
        epsilon = epsilon*0.99       # decrease epsilon, decrease randomness!
        if show:
            env.render()
    if total_reward>=best_reward:      # save the best W 
        best_reward = total_reward
        best_W = sess.run(W)
    clear_output(wait=True)                    # clear output every episode and replace with new (for jupyter notebook)
    print("episode: " + str(episode))          # print current episode            
    print("epsilon: " + str(epsilon))          # print current epsilon
    print("total rewards of last episode: " + str(total_reward))           # total rewards  of  last epsode
    print("best reward: " + str(best_reward))                              # best reward till now
    print("W: ")                                                           # enjoy watching while the optimizer tweaks your W matrix
    print(sess.run(W))
            
env.close()   

episode: 499
epsilon: 1.7216336019101414e-105
total rewards of last episode: 23.0
best reward: 498.0
W: 
[[ 0.27603316  0.08331281  0.31661391  0.00605107]
 [-0.136452   -0.05279967  0.03332765 -0.00640894]]


In [6]:
assign =  W.assign(best_W)
sess.run(assign)

done = False
total_reward = 0
new_state = env.reset()
while not done:
        action  = sess.run(Action, feed_dict = {New_state:new_state.reshape((4,1))})
        new_state, reward, done, _ = env.step(action[0])
        total_reward += reward
        env.render()
        
print(total_reward)
env.close()

500.0
